### EPA Water Quality Data for CA

In [1]:
import requests
from io import StringIO
import pandas as pd
from datetime import timedelta
import os
import sys
import logging
from sodapy import Socrata
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import langchain
from langchain.agents import initialize_agent, Tool, AgentType, AgentExecutor
from langchain.llms import OpenAI, OpenAIChat, HuggingFaceHub
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory, ConversationBufferWindowMemory, ConversationSummaryMemory, ConversationSummaryBufferMemory
from langchain.chains import LLMChain
from langchain.prompts import MessagesPlaceholder





ModuleNotFoundError: No module named 'sodapy'

In [ ]:

# Use a known-valid site in California (e.g., USGS-11447650)
BASE_URL = "https://www.waterqualitydata.us/data/Result/search"
params = {
    "siteid": "USGS-11447650",
    "mimeType": "csv",
    "zip": "no"
}

response = requests.get(BASE_URL, params=params)
print(f"Status code: {response.status_code}")
print(response.text[:500])  # Check for content

# Load into DataFrame if there's content
if response.text.strip():  # Make sure it's not empty
    df = pd.read_csv(StringIO(response.text))
    print(df.head())
else:
    print("⚠️ No data returned.")

### List of stations in CA

In [ ]:
STATION_URL = "https://www.waterqualitydata.us/data/Station/search"

params = {
    "statecode": "US:06",   # California
    "mimeType": "csv",
    "zip": "no"
}

response = requests.get(STATION_URL, params=params)
print(f"Status code: {response.status_code}")
print(response.text[:1000])

Status code: 200
OrganizationIdentifier,OrganizationFormalName,MonitoringLocationIdentifier,MonitoringLocationName,MonitoringLocationTypeName,MonitoringLocationDescriptionText,HUCEightDigitCode,DrainageAreaMeasure/MeasureValue,DrainageAreaMeasure/MeasureUnitCode,ContributingDrainageAreaMeasure/MeasureValue,ContributingDrainageAreaMeasure/MeasureUnitCode,LatitudeMeasure,LongitudeMeasure,SourceMapScaleNumeric,HorizontalAccuracyMeasure/MeasureValue,HorizontalAccuracyMeasure/MeasureUnitCode,HorizontalCollectionMethodName,HorizontalCoordinateReferenceSystemDatumName,VerticalMeasure/MeasureValue,VerticalMeasure/MeasureUnitCode,VerticalAccuracyMeasure/MeasureValue,VerticalAccuracyMeasure/MeasureUnitCode,VerticalCollectionMethodName,VerticalCoordinateReferenceSystemDatumName,CountryCode,StateCode,CountyCode,AquiferName,LocalAqfrName,FormationTypeText,AquiferTypeName,ConstructionDateText,WellDepthMeasure/MeasureValue,WellDepthMeasure/MeasureUnitCode,WellHoleDepthMeasure/MeasureValue,WellHoleDep

In [ ]:
RESULT_URL = "https://www.waterqualitydata.us/data/Result/search"

params = {
    "siteid": "USGS-11447650",   # A known valid station ID from California
    "mimeType": "csv",
    "zip": "no"
}

response = requests.get(RESULT_URL, params=params)
print(f"Status code: {response.status_code}")
print(response.text[:1000])

Status code: 200
OrganizationIdentifier,OrganizationFormalName,ActivityIdentifier,ActivityTypeCode,ActivityMediaName,ActivityMediaSubdivisionName,ActivityStartDate,ActivityStartTime/Time,ActivityStartTime/TimeZoneCode,ActivityEndDate,ActivityEndTime/Time,ActivityEndTime/TimeZoneCode,ActivityDepthHeightMeasure/MeasureValue,ActivityDepthHeightMeasure/MeasureUnitCode,ActivityDepthAltitudeReferencePointText,ActivityTopDepthHeightMeasure/MeasureValue,ActivityTopDepthHeightMeasure/MeasureUnitCode,ActivityBottomDepthHeightMeasure/MeasureValue,ActivityBottomDepthHeightMeasure/MeasureUnitCode,ProjectIdentifier,ActivityConductingOrganizationText,MonitoringLocationIdentifier,ActivityCommentText,SampleAquifer,HydrologicCondition,HydrologicEvent,SampleCollectionMethod/MethodIdentifier,SampleCollectionMethod/MethodIdentifierContext,SampleCollectionMethod/MethodName,SampleCollectionEquipmentName,ResultDetectionConditionText,CharacteristicName,ResultSampleFractionText,ResultMeasureValue,ResultMeasure/

### SF Excavation Permits Dataset

In [ ]:
# Initialize Socrata client for San Francisco's open data portal
client = Socrata("data.sfgov.org", None)

# Dataset ID for Utility Excavation Permits
dataset_id = "smdf-6c45"

# Fetch the latest 1000 records
results = client.get(dataset_id, limit=1000)

# Convert to DataFrame
df = pd.DataFrame.from_records(results)

# Display relevant fields
print(df.columns.tolist())

['permit_number', 'streetname', 'cross_street_1', 'cross_street_2', 'utility_contractor', 'permit_reason', 'utility_type', 'effective_date', 'expiration_date', 'status', 'cnn']


## ===================

In [ ]:
# For sensor data
df_results["ActivityStartDate"] = pd.to_datetime(df_results["ActivityStartDate"])

# For permit data
df_permits["effective_date"] = pd.to_datetime(df_permits["effective_date"], errors='coerce')
df_permits["expiration_date"] = pd.to_datetime(df_permits["expiration_date"], errors='coerce')

In [ ]:
# Outer loop to find relevant permits per sensor reading
def find_matching_permits(sensor_row, permits_df):
    matches = permits_df[
        (permits_df["effective_date"] <= sensor_row["ActivityStartDate"]) &
        (permits_df["expiration_date"] >= sensor_row["ActivityStartDate"])
    ]
    return matches

# Apply logic
sensor_with_permits = []
for _, row in df_results.iterrows():
    matching_permits = find_matching_permits(row, df_permits)
    for _, permit in matching_permits.iterrows():
        sensor_with_permits.append({
            "timestamp": row["ActivityStartDate"],
            "characteristic": row["CharacteristicName"],
            "value": row["ResultMeasureValue"],
            "unit": row["ResultMeasure/MeasureUnitCode"],
            "permit_number": permit["permit_number"],
            "permit_reason": permit["permit_reason"],
            "street": permit["streetname"],
            "contractor": permit["utility_contractor"]
        })

# Create joined DataFrame
df_joined = pd.DataFrame(sensor_with_permits)

In [ ]:
def search_permits_by_date(target_date, permits_df, window_days=3):
    # Look +/- window_days around the event
    start = target_date - timedelta(days=window_days)
    end = target_date + timedelta(days=window_days)
    
    matches = permits_df[
        (permits_df["effective_date"] <= end) & (permits_df["expiration_date"] >= start)
    ]
    return matches.to_dict(orient="records")

In [ ]:
def search_sensor_anomalies(characteristic_name, df_sensor, z_threshold=2.0):
    """
    Find anomalies in the sensor dataset based on a simple z-score method.
    
    Args:
        characteristic_name (str): e.g., 'Dissolved oxygen (DO)', 'Water temperature'
        df_sensor (DataFrame): Sensor data with columns: 'CharacteristicName', 'ResultMeasureValue', 'ActivityStartDate'
        z_threshold (float): How many standard deviations from mean counts as anomaly (default = 2.0)

    Returns:
        list of dicts: Anomalous records
    """
    # Filter by characteristic
    df_filtered = df_sensor[df_sensor["CharacteristicName"].str.contains(characteristic_name, case=False, na=False)].copy()

    if df_filtered.empty:
        return [{"message": f"No data found for characteristic: {characteristic_name}"}]

    # Convert result to numeric, handle missing values
    df_filtered["ResultMeasureValue"] = pd.to_numeric(df_filtered["ResultMeasureValue"], errors="coerce")
    df_filtered = df_filtered.dropna(subset=["ResultMeasureValue"])

    # Calculate mean and std
    mean = df_filtered["ResultMeasureValue"].mean()
    std = df_filtered["ResultMeasureValue"].std()

    # Define upper and lower bounds
    upper = mean + z_threshold * std
    lower = mean - z_threshold * std

    # Find anomalies
    anomalies = df_filtered[(df_filtered["ResultMeasureValue"] > upper) | (df_filtered["ResultMeasureValue"] < lower)]

    # Return a list of anomaly records
    return anomalies[[
        "ActivityStartDate", 
        "CharacteristicName", 
        "ResultMeasureValue", 
        "ResultMeasure/MeasureUnitCode"
    ]].to_dict(orient="records")

In [ ]:
# Create LLM
llm = OpenAI(temperature=0)

# Create tools
tools = [
    Tool(
        name="SearchPermitsByDate",
        func=lambda date: search_permits_by_date(pd.to_datetime(date), df_permits),
        description="Useful for finding permits active near a specific date"
    ),
    Tool(
        name="SearchSensorAnomalies",
        func=lambda characteristic: search_sensor_anomalies(characteristic, df_results),
        description="Useful for finding anomalies (like low oxygen or pressure events)"
    )
]

# Initialize agent
agent = initialize_agent(
    tools,
    llm,
    agent="zero-shot-react-description",  # Standard "think-act-observe" loop
    verbose=True
)

# Example prompt
query = "Were there any underground permits active when the oxygen levels dropped near USGS-11447650 last year?"
agent.run(query)

In [ ]:
# Set your HuggingFace token
os.environ["HUGGINGFACEHUB_API_TOKEN"] = "your_huggingface_token_here"

# Load model directly from HuggingFace Hub
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.3")
model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.3")
)

In [ ]:
messages = [
    {"role": "user", "content": "Who are you?"},
]
pipe = pipeline("text-generation", model="mistralai/Mistral-7B-Instruct-v0.3")
pipe(messages)


In [ ]:
# Define your tools (same as before)
tools = [
    Tool(
        name="SearchPermitsByDate",
        func=lambda date: search_permits_by_date(pd.to_datetime(date), df_permits),
        description="Useful for finding permits active near a specific date"
    ),
    Tool(
        name="SearchSensorAnomalies",
        func=lambda characteristic: search_sensor_anomalies(characteristic, df_results),
        description="Useful for finding anomalies (like low oxygen or pressure events)"
    )
]

# Initialize the agent
agent = initialize_agent(
    tools,
    llm,
    agent="zero-shot-react-description",  # Core reasoning agent
    verbose=True
)

# Now run it!
query = "Were there any sewer repair permits near the oxygen drop at USGS-11447650 last March?"
agent.run(query)